In [3]:
from pydantic_settings import BaseSettings

class Setting(BaseSettings):
    FRED_API_KEY: str

    class Config:
        env_file = "../../.env"
        env_file_encoding = "utf-8"

setting = Setting()

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gemma3:12b", temperature=0)

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """
        당신은 경제 전문가 입니다.
        사용자의 질문에 답을 더욱 잘 하기 위해 필요한 자료를 요청해야 합니다.
        사용자가 제공할 수 있는 정보는 FRED에서 가져온 UNRATE, DTWEXM 정보 입니다.
        어느 정보가 필요한가요?
        다른 답변을 추가하지 않고, 아래와 같은 JSON 형식으로 답변을 제공해주세요.
        {{
            "data_type": "UNRATE or DTWEXM",
            "start_date": "yyyy-mm-dd",
            "end_date": "yyyy-mm-dd"
        }}
    """),
    ("human", """
    오늘은 {today} 입니다.
    {question}\n
    """)
])

In [13]:
from datetime import datetime

chain = prompt | llm

ai_response = chain.invoke({
    "question": "미국 실업률의 추이를 알려주세요.",
    "today": datetime.today().strftime("%Y-%m-%d")
})

print(ai_response)
print(type(ai_response))

content='```json\n{\n    "data_type": "UNRATE",\n    "start_date": "2023-03-17",\n    "end_date": "2025-03-17"\n}\n```' additional_kwargs={} response_metadata={'model': 'gemma3:12b', 'created_at': '2025-03-17T07:25:49.34109Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13896335667, 'load_duration': 6826781667, 'prompt_eval_count': 175, 'prompt_eval_duration': 3273000000, 'eval_count': 58, 'eval_duration': 3779000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-2c49923a-e7c9-48d4-b316-c0c723857604-0' usage_metadata={'input_tokens': 175, 'output_tokens': 58, 'total_tokens': 233}
<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
import json
import re


def extract_json_from_llm_response(response_text):
    json_pattern = r'```json\n(.*?)\n```'
    match = re.search(json_pattern, response_text, re.DOTALL)

    if match:
        json_str = match.group(1)
        return json.loads(json_str)
    else:
        raise ValueError("응답에서 JSON을 찾을 수 없습니다")


llm_response = extract_json_from_llm_response(ai_response.content)

date_type = llm_response["data_type"]
start_date = llm_response["start_date"]
end_date = llm_response["end_date"]


In [8]:
url = f'https://api.stlouisfed.org/fred/series/observations'

series_id = "UNRATE"
frequency = "m"


params = {
    'series_id': date_type,
    'api_key': setting.FRED_API_KEY,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': frequency
}

In [9]:
import requests

response = requests.get(url, params=params)
response.json()["observations"]

[{'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-03-01',
  'value': '3.5'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-04-01',
  'value': '3.4'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-05-01',
  'value': '3.6'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-06-01',
  'value': '3.6'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-07-01',
  'value': '3.5'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-08-01',
  'value': '3.7'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-09-01',
  'value': '3.8'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-10-01',
  'value': '3.9'},
 {'realtime_start': '2025-03-17',
  'realtime_end': '2025-03-17',
  'date': '2023-11-01',
  'value': '3.7'},
 {'realtime_start':

In [10]:
answer_llm = ChatOllama(model="gemma3:12b", temperature=0.7)

answer_prompt = ChatPromptTemplate.from_messages([
    ("system", """
        당신은 경제 전문가 입니다.
        아래 사용자 질문과 사전 정보를 갖고 답변을 작성해주세요.
    """),
    ("human", """
    오늘은 {today} 입니다.
    사전 정보: {data}
    사용자 질문: {question}
    """)
])

answer_chain = answer_prompt | answer_llm


In [11]:
ai_answer = answer_chain.invoke({
    "question": "미국 실업률의 추이를 알려주세요",
    "data": response.json()["observations"],
    "today": datetime.today().strftime("%Y-%m-%d")
})

In [12]:
ai_answer.content

'2023년 3월부터 2025년 2월까지의 미국 실업률 추이는 다음과 같습니다.\n\n*   **2023년 3월:** 3.5%\n*   **2023년 4월:** 3.4%\n*   **2023년 5월:** 3.6%\n*   **2023년 6월:** 3.6%\n*   **2023년 7월:** 3.5%\n*   **2023년 8월:** 3.7%\n*   **2023년 9월:** 3.8%\n*   **2023년 10월:** 3.9%\n*   **2023년 11월:** 3.7%\n*   **2023년 12월:** 3.8%\n*   **2024년 1월:** 3.7%\n*   **2024년 2월:** 3.9%\n*   **2024년 3월:** 3.9%\n*   **2024년 4월:** 3.9%\n*   **2024년 5월:** 4.0%\n*   **2024년 6월:** 4.1%\n*   **2024년 7월:** 4.2%\n*   **2024년 8월:** 4.2%\n*   **2024년 9월:** 4.1%\n*   **2024년 10월:** 4.1%\n*   **2024년 11월:** 4.2%\n*   **2024년 12월:** 4.1%\n*   **2025년 1월:** 4.0%\n*   **2025년 2월:** 4.1%\n\n전반적으로 실업률은 2023년 초 3.4%로 최저점을 찍은 후 2024년까지 완만하게 상승하여 4.1%까지 증가했습니다. 2025년 2월에는 4.1%로 마무리되었습니다.'